In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

from IPython.display import clear_output

- Concateno toda la info de educacion

In [2]:
edu_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')])
edu_totales.drop_duplicates(inplace=True)

edu_totales = edu_totales.rename(columns={'nombre':'estudios'})
edu_totales['estudios'] = edu_totales['estudios'].astype('category')
edu_totales['estado'] = edu_totales['estado'].astype('category')

edu_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 397508
Data columns (total 3 columns):
idpostulante    693641 non-null object
estudios        693641 non-null category
estado          693641 non-null category
dtypes: category(2), object(1)
memory usage: 11.9+ MB


In [3]:
# Creo nuevas columnas indicando el nivel del estudio y del estado, para filtrar mas facilmente luego

edu_estado_list = ['Abandonado','En Curso', 'Graduado']
edu_estudios_list = ['Otro', 'Secundario','Terciario/Técnico', 'Universitario', 'Posgrado', 'Master', 'Doctorado']

edu_totales['nivel_estado'] = edu_totales['estado'].apply(lambda x: edu_estado_list.index(x)).astype('int8')
edu_totales['nivel_estudios'] = edu_totales['estudios'].apply(lambda x: edu_estudios_list.index(x)).astype('int8')

edu_totales.head()

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
0,NdJl,Posgrado,En Curso,1,4
1,8BkL,Universitario,En Curso,1,3
2,1d2B,Universitario,En Curso,1,3
3,NPBx,Universitario,En Curso,1,3
4,NPBx,Master,En Curso,1,5


In [4]:
edu_totales[edu_totales['idpostulante']=='5vmKGM']

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
48497,5vmKGM,Universitario,En Curso,1,3
50630,5vmKGM,Terciario/Técnico,Graduado,2,2
51662,5vmKGM,Universitario,Abandonado,0,3
51663,5vmKGM,Terciario/Técnico,Abandonado,0,2
169914,5vmKGM,Doctorado,Graduado,2,6


In [5]:
#Ordeno por niveles de importancia, primero por estado, luego por estudio

edu_totales = edu_totales.sort_values(['nivel_estado','nivel_estudios'], ascending=[True, True])
edu_totales[edu_totales['idpostulante']=='5vmKGM']

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
51663,5vmKGM,Terciario/Técnico,Abandonado,0,2
51662,5vmKGM,Universitario,Abandonado,0,3
48497,5vmKGM,Universitario,En Curso,1,3
50630,5vmKGM,Terciario/Técnico,Graduado,2,2
169914,5vmKGM,Doctorado,Graduado,2,6


In [6]:
#Ante duplicados me quedo con el de nivel de importancia de estado mas alto, y dentro de ellos, con el de nivel de estudio mas alto. Esto es para evitar borrar 
#datos de personas que solo tengan estudios en curso o abandonados.

edu_totales = edu_totales.drop_duplicates(subset=['idpostulante'], keep='last')
edu_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 7621 to 385512
Data columns (total 5 columns):
idpostulante      447909 non-null object
estudios          447909 non-null category
estado            447909 non-null category
nivel_estado      447909 non-null int8
nivel_estudios    447909 non-null int8
dtypes: category(2), int8(2), object(1)
memory usage: 8.5+ MB


In [7]:
edu_totales[edu_totales['idpostulante']=='5vmKGM']

,idpostulante,estudios,estado,nivel_estado,nivel_estudios
169914,5vmKGM,Doctorado,Graduado,2,6


In [8]:
edu_totales.drop(columns=['nivel_estado','nivel_estudios'], inplace=True)

In [9]:
edu_totales.sort_values(by=['idpostulante'], ascending=True, inplace=True)
edu_totales.reset_index(drop=True, inplace=True)
edu_totales.head()

,idpostulante,estudios,estado
0,0z5Dmrd,Universitario,Graduado
1,0z5JW1r,Terciario/Técnico,Graduado
2,0z5VvGv,Secundario,Graduado
3,0zB01pE,Terciario/Técnico,Graduado
4,0zB026d,Terciario/Técnico,Graduado


- Concateno todas las postulaciones

In [10]:
post_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')])
post_totales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    object
dtypes: int64(1), object(2)
memory usage: 253.6+ MB


In [11]:
post_totales.drop_duplicates(inplace=True)
post_totales['fechapostulacion'] = pd.to_datetime(post_totales['fechapostulacion'])
post_totales.sort_values(by=['fechapostulacion'], ascending=True, inplace=True)
post_totales.drop_duplicates(subset=['idaviso','idpostulante'], keep='first', inplace=True)
post_totales.sort_values(by=['idpostulante'], ascending=True, inplace=True)
post_totales.reset_index(drop=True, inplace=True)
post_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 151.1+ MB


In [12]:
post_totales.head()

,idaviso,idpostulante,fechapostulacion
0,1112384041,0z5Dmrd,2018-03-28 16:24:37
1,1112420060,0z5Dmrd,2018-03-28 16:47:11
2,1112428361,0z5JW1r,2018-04-01 07:26:12
3,1112309728,0z5JW1r,2018-03-02 03:49:53
4,1112292169,0z5JW1r,2018-02-19 05:12:24


- Calculo para cada aviso cuantos postulantes hubo, para realizar las no-postulaciones con las mismas cantidades por aviso, para que los sets de post y no-post sean balanceados

In [ ]:
post_por_avisos = post_totales.groupby(by='idaviso').count()
post_por_avisos.drop(columns='fechapostulacion', inplace=True)
post_por_avisos.rename(columns={'idpostulante':'count'},inplace=True)
post_por_avisos.sort_values(by=['count'], ascending=False, inplace=True)
post_por_avisos.head(5)

,count
idaviso,
1112334791,24420
1112094756,22424
1112033906,20570
1112334788,16687
1112196813,13619


- Calculo el set aleatorio de no-postulaciones, con las cantidades de usuarios por aviso calculadas antes

In [ ]:
users_id = edu_totales['idpostulante'].to_frame()

users_sample = pd.DataFrame(columns=['idpostulante','idaviso'])
no_post_totales = pd.DataFrame(columns=['idpostulante','idaviso'])
post_totales_reduced = pd.DataFrame(columns=['idpostulante','idaviso']) 
available_users = pd.DataFrame(columns=['idpostulante'])

progress=0
progress_size = post_por_avisos.count()
success = 0

for index, row in post_por_avisos.iterrows():

    progress += 1
    success = 0
    post_totales_reduced = (post_totales[post_totales['idaviso']==index])[['idpostulante', 'idaviso']]
    
    if(row['count']>600):
    
        available_users = pd.concat([users_id, (pd.merge(users_id, post_totales_reduced['idpostulante'].to_frame()))]).drop_duplicates(keep=False)

        users_sample = available_users.sample(n=row['count'])
        users_sample['idaviso'] = index

    else:

        while(success == 0):

            users_sample = users_id.sample(n=row['count'])
            users_sample['idaviso'] = index

            if((pd.merge(post_totales_reduced, users_sample).size) == 0):
                success = 1
            
    no_post_totales = pd.concat([no_post_totales, users_sample])
    clear_output(wait=True)
    print('remaining:', np.int(progress_size-progress), '- total progress:', np.float((progress/progress_size)*100), '%')

remaining: 19988 - total progress: 1.2011269833423952 %


In [ ]:
no_post_totales['fechapostulacion'] = ''
no_post_totales.head()

In [ ]:
#no_post_totales.to_csv('no_post_totales.csv', index=False)